# Bayesian Addition of Astronomical Data: Formalism

### Overview

Combine individual observations of 1D or 2D binned data that satisfy the following conditions:
 - Each observation views the same static source.
 - Each observation is convolved with a known (possibly varying) point-spread function.
 - Observed data is accumulated into bins (pixels) with known edges on a regular (not necessarily uniform) grid.
 - Statistical uncertainties in bin values are well described by a known Gaussian
   inverse covariance matrix (not necessarily diagonal).
 - Missing or bad data is flagged with zero inverse variance.

The joint likelihood of all observations is accumulated using the sufficient summary
statistics proposed in Kaiser 2004 "Addition of Images with Varying Seeing" (unpublished
technical note). Observations are then combined in a Bayesian framework with a Gaussian
prior having a single hyperparameter. (An equivalent view of this procedure is that the
hyperparameter regularizes the extraction of high-frequency information that has been
erased by PSF convolution.) Methods are provided to calculate the data evidence
as a function of this hyperparameter, and to support model optimization or averaging.

An extracted "coadd" takes the form of a multivariate Gaussian posterior probability
density (specifed with a mean vector and covariance matrix) in parameters that are
arbitrary linear combinations of the true flux tabulated on a high-resolution grid.
Convenience methods are provided for:
 - Estimated true flux downsampled to square pixels.
 - Estimated true flux convolved with a Gaussian PSF of fixed size.
 - Estimated true flux convolved with an effective PSF that whitens the noise.

Extracted coadds are numerically stable and well defined within a Bayesian framework
for any sequence of observations.  With a suitable choice of extracted pixel or
Gaussian PSF width, the results are also insensitive to the choice of hyperparameter.

### Observation Inputs

Each observation $D_j$, indexed by $j$, consists of $n_j$ pixels.  The formalism for 1D and 2D data is essentially the same once we "flatten" the row and column indices for 2D pixels down to a single data-vector index $i$.

The observed pixels are specified by:
 - A vector $\mathbf{d}_j$ of $n_j$ flux estimates,
 - An $n_j\times n_j$ covariance matrix $C_j$ of (possibly correlated) statistical uncertainties in these flux estimates,
 - An $n_j\times n_j$ matrix of calibration factors $F_j$ to account for varying exposure time, throughput, crosstalk, etc,

The pixel boundaries lie on a known regular grid:
 - 1D: boundaries are specified by an array of increasing edge values (usually in wavelength).
 - 2D: boundaries are specified by separate row and column arrays of increasing edge values (usually in length or angular offsets).

The PSF is "known", meaning that we can tabulate the support $\mathbf{g}_{ij}$ of each observed pixel $i$ in each observation $j$ as the vector of probabilities that true flux disperses into the pixel from each point on a high-resolution grid.  If we assume that the PSF varies slowly between neighboring pixels, then we can approximate this support as the convolution of each pixel's "top-hat" response with the PSF at its center. The high-resolution grid has a (flattened) size $N \gg n_i$, that extends beyond all observations, allowing for dispersion by the largest expected PSF. In practice, tabulated PSFs are truncated and renormalized.

### Probabilistic Model

#### Likelihood

Our $N$ model parameters $\mathbf{f}$ are the true (i.e., calibrated and undispersed) source fluxes tabulated on the same high-resolution grid as the per-pixel PSFs.  The predicted mean flux for observation $j$ is then:
$$
\langle \mathbf{d}_j\rangle = F_j G_j\,\mathbf{f}
$$
where $G_j$ is the $n_j\times N$ matrix whose rows are the support vectors $\mathbf{g}_{ij}$.

The log-likelihood of observation $j$ is then
$$
\log P(D_j \mid \mathbf{f}) = -\frac{n_j}{2} \log(2\pi) -\frac{1}{2}\log|C_j|
-\frac{1}{2} \left(\mathbf{d}_j - F_j G_j\,\mathbf{f}\right)^t C_j^{-1} \left(\mathbf{d}_j - F_j G_j\,\mathbf{f}\right) \; .
$$
A dataset $D$ consisting of $M$ independent observations indexed by $j$, has the joint likelihood
$$
\log P(D \mid \mathbf{f}) = -\frac{\sum_j n_j}{2} \log(2\pi) -\frac{1}{2} \sum_{j=1}^M\left[
\log|C_j| + 
\left(\mathbf{d_j} - F_j G_j\,\mathbf{f}\right)^t C_j^{-1} \left(\mathbf{d_j} - F_j G_j\,\mathbf{f}\right)
\right]\; .
$$

#### Prior

Chose a Gaussian prior on the parameters $\mathbf{f}$,
$$
\log P(\mathbf{f}\mid \sigma) = -\frac{N}{2}\log(2\pi) -N \log\sigma -\frac{1}{2} \sigma^{-2} \mathbf{f}^t \mathbf{f} \; ,
$$
where $\sigma$ is a hyperparameter of the prior.  This prior states that, in the absence of any observations, the expected source flux is zero with a variance $\sigma^2$ per high-resolution grid element.  This is not particularly physically motivated but serves as an effective regularization of an ill-conditioned extraction.  Extracted results should normally not depend on the choice of prior.

#### Summary Statistics

Combine the likelihood and prior to evaluate the joint probability density over the data and parameters:
$$
\begin{aligned}
\log P(D, \mathbf{f}\mid \sigma) &= \log P(D\mid \mathbf{f}) + \log P(\mathbf{f}\mid \sigma) \\
&=
-\frac{\sum_j n_j+N}{2}\log(2\pi) - N \log\sigma
-\frac{1}{2} \sum_{j=1}^M \log|C_j|\\
&\quad -\frac{1}{2}\sigma^{-2} \mathbf{f}^t \mathbf{f}
-\frac{1}{2} \sum_{j=1}^M
\left(\mathbf{d_j} - F_j G_j\,\mathbf{f}\right)^t C_j^{-1} \left(\mathbf{d_j} - F_j G_j\,\mathbf{f}\right) \\
&=
-\frac{\sum_j n_j+N}{2}\log(2\pi) - N \log\sigma
-\frac{1}{2} \sum_{j=1}^M \log|C_j|\\
&\quad -\frac{1}{2} \sum_{j=1}^M \mathbf{d_j}^t C_j^{-1} \mathbf{d_j}
-\frac{1}{2} \mathbf{f}^t A(\sigma)\,\mathbf{f}
+\mathbf{f}^t \boldsymbol{\varphi} \\
&=
-\frac{\sum_j n_j+N}{2}\log(2\pi) - N \log\sigma
-\frac{1}{2} \sum_{j=1}^M \log|C_j|\\
&\quad -\frac{1}{2} \sum_{j=1}^M \mathbf{d_j}^t C_j^{-1} \mathbf{d_j}
-\frac{1}{2} \left( \mathbf{f} - A(\sigma)^{-1}\boldsymbol{\varphi}\right)^t A(\sigma)
\left( \mathbf{f} - A(\sigma)^{-1}\boldsymbol{\varphi}\right)
+\frac{1}{2}\boldsymbol{\varphi}^t A(\sigma)^{-1}\,\boldsymbol{\varphi}
\end{aligned}
$$
with the contributions of the individual exposures only appearing in the summary statistics
$$
A(\sigma)\equiv \sum_{j=1}^M G_j^t F_j^t C_j^{-1} F_j G_j + \sigma^{-2}\mathbb{1}_N \quad , \quad
\boldsymbol{\varphi} \equiv \sum_{j=1}^M G_j^t F_j^t C_j^{-1} \mathbf{d_j} \; .
$$
These two quantities are [sufficient statistics](https://en.wikipedia.org/wiki/Sufficient_statistic) since they contain all the information about the individual exposures necessary to evaluate the full joint likelihood (and therefore also the posterior). The limited support $\mathbf{g}_{ij}$ of each pixel leads to a sparse structure for the $N\times N$ matrix $A(\sigma)$, with a fraction of non-zero elements $\simeq N_{\rm PSF} / N$ where $N_{\rm PSF}$ is the typical number of non-zero elements in each $\mathbf{g}_{ij}$.  The (much smaller) vector $\boldsymbol{\varphi}$ is generally dense.

#### Evidence and Posterior

Integrate over $\mathbf{f}$ to calculate the log-evidence given the hyperparameter $\sigma$
$$
\begin{aligned}
\log P(D\mid\sigma) &= \log \int P(D, \mathbf{f}\mid\sigma)\,d\,\mathbf{f} \\
&= -\frac{\sum_j n_j+2N}{2}\log(2\pi)
-\frac{1}{2} \sum_{j=1}^M \mathbf{d_j}^t C_j^{-1} \mathbf{d_j}
-\frac{1}{2} \sum_{j=1}^M \log|C_j|
-N \log\sigma
-\frac{1}{2}\log|A(\sigma)|
+\frac{1}{2}\boldsymbol{\varphi}^t A(\sigma)^{-1}\,\boldsymbol{\varphi} \\
&= \text{constant} -N \log\sigma -\frac{1}{2}\log|A(\sigma)|
+\frac{1}{2}\boldsymbol{\varphi}^t A(\sigma)^{-1}\,\boldsymbol{\varphi}
\; ,
\end{aligned}
$$
where only the last three terms depend on $\sigma$. The normalized log-posterior is then
$$
\begin{aligned}
\log P(\mathbf{f}\mid D, \sigma) &= \log P(D\mid \mathbf{f}) + \log P(\mathbf{f}\mid \sigma) - \log P(D\mid\sigma) \\
&= -\frac{N}{2}\log(2\pi) + \frac{1}{2}\log |A(\sigma)|
-\frac{1}{2} \left( \mathbf{f} - A(\sigma)^{-1}\boldsymbol{\varphi}\right)^t A(\sigma)
\left( \mathbf{f} - A(\sigma)^{-1}\boldsymbol{\varphi}\right) \; .
\end{aligned}
$$
Empirical studies show that the most probable $\sigma$ scales with the high-resolution grid spacing $\propto N^{-1}$ but only mildy with the data pixel size and uncertainty, or the number of exposures in the coadd.

### Extraction

The formal solution to the maximum-likelihood (ML) problem,
$$
\mathbf{f}_{\rm ML} = A(0)^{-1} \boldsymbol{\varphi} \; ,
$$
is generally ill-conditioned since it uses a deconvolution to attempt to reconstruct high-frequency information that has been erased by the PSFs. On the other hand, the maximum-a-posteori (MAP) problem has a well behaved solution,
$$
\mathbf{f}_{\rm MAP} = A(\sigma)^{-1} \boldsymbol{\varphi} \; ,
$$
in which the hyperparameter $\sigma$ effectively regularizes the ML problem, but biases the high-frequency reconstruction.  Note that a sparse $A(\sigma)$ does not translate into a sparse inverse $A(\sigma)^{-1}$, so it is generally more efficient to obtain $\mathbf{f}$ as the solution to the sparse linear system
$$
A(\sigma)\,\mathbf{f}_{\rm MAP} = \boldsymbol{\varphi} \; .
$$

#### Downsampling

Rather than estimate $\mathbf{f}$, we can perform a linear change of variables
$$
\mathbf{h} = H \mathbf{f}
$$
to extract $P < N$ parameters that filter out the poorly constrained high-frequency information, where $H$ has dimensions $P\times N$. An estimation of $\mathbf{h}$ should then be insensitive to the choice of $\sigma$. For example, $H$ could implement downsampling to a predefined pixel grid or convolution with a predefined (truncated) PSF.

To perform this change of variables, we first embed $H$ in an $N\times N$ square invertible matrix $K$ as:
$$
K_{ij} = \begin{cases}
H_{k_p j} & p = 1,\ldots, P\\
\delta_{ij} & \rm{otherwise}
\end{cases}
$$
In other words, $K$ is an identity matrix with row $k_p$ replaced by row $p$ of $H$. The $k_p$ must each be different and chosen so that $K$ is invertible.  In practice, setting
$$
k_p = \underset{i}{\operatorname{argmax}} H_{pi}
$$
works in most cases.

The log-posterior PDF $P_k$ for $\mathbf{k} = K \mathbf{f}$ is then
$$
\begin{aligned}
\log P_k(\mathbf{k}\mid D, \sigma) &= \log |K^{-1}| + \log P(\mathbf{f} = K^{-1}\mathbf{k} \mid D, \sigma) \\
&= -\frac{N}{2}\log(2\pi) + \frac{1}{2}\log \left|K^{-t} A(\sigma) K^{-1}\right|
-\frac{1}{2} \left( \mathbf{k} - (A(\sigma) K^{-1})^{-1}\boldsymbol{\varphi}\right)^t K^{-t} A(\sigma) K^{-1}
\left( \mathbf{k} - (A(\sigma) K^{-1})^{-1}\boldsymbol{\varphi}\right) \; .
\end{aligned}
$$

Therefore the mean value $\langle\mathbf{k}\rangle$ is the solution to
$$
A(\sigma) K^{-1}\,\mathbf{k} = \boldsymbol{\varphi}
$$
and $\langle\mathbf{h}\rangle$ are its elements at each $k_p$.  We calculate the covariance $C_h$ of the estimated $\mathbf{h}$ by marginalizing over the $N-P$ nuiscance parameters of $\mathbf{k}$. Since the posterior is Gaussian in $\mathbf{k}$, this simply involves restricting the full $\mathbf{k}$ covariance,
$$
C_k = K A(\sigma)^{-1} K^t
$$
to the rows and columns at each $k_p$.  In practice, we obtain $C_k$ indirectly as the solution to the linear system
$$
A(\sigma) K^{-1} C_k = K^t \; .
$$

Note that the summary statistics, $A(\sigma)$ and $\boldsymbol{\varphi}$, and downsampling scheme, $H$, only enter the calculation of $\langle\mathbf{k}\rangle$ and $C_k$ through the combination $A(\sigma) K^{-1}$. To evaluate this combination efficiently, we note that the rows and columns of $K$ can be permuted so that it has the block diagonal form
$$
\begin{bmatrix}
H_1 & H_2 \\
\mathbb{0}_{(N-P)\times P} & \mathbb{1}_{N-P}
\end{bmatrix}
$$
with inverse
$$
\begin{bmatrix}
H_1^{-1} & -H_1^{-1} H_2 \\
\mathbb{0}_{(N-P)\times P} & \mathbb{1}_{N-P}
\end{bmatrix} \; ,
$$
where $H_1$ and $H_2$ partition the columns of the original $H$ and the sparse matrix $H_1$ will generally have a sparse inverse $H_1^{-1}$.

#### Hyperparameter Selection

Ideally, any extracted results are insensitive to the choice of hyperparameter $\sigma$.  Any sensitivity can be empirically measured by performing several extractions with different hyperparameters. When there is some residual sensitivity to $\sigma$, there are two options: use the value preferred by the data (as measured by the evidence $P(D\mid \sigma)$, or marginalize over the space of models.

The first approach involves find a solution (with standard root-finding numerical methods) to
$$
\frac{d}{d\log\sigma} \log P(D\mid \sigma) = 0 \; ,
$$
where we can simplify
$$
\begin{aligned}
\frac{d}{d\log\sigma} \log P(D\mid \sigma) &=
\sigma \frac{d}{d\sigma} \log P(D\mid \sigma) \\
&= -N - \frac{\sigma}{2|A(\sigma)|} \frac{d}{d\sigma}|A(\sigma)|
+ \frac{\sigma}{2} \boldsymbol{\varphi}^t \frac{d}{d\sigma} A^{-1}(\sigma) \boldsymbol{\varphi} \\
&= -N - \frac{\sigma}{2} \text{tr}\left( A^{-1}(\sigma) \frac{d}{d\sigma} A(\sigma)\right)
-\frac{\sigma}{2} \boldsymbol{\varphi}^t A^{-1}(\sigma) \frac{d}{d\sigma} A(\sigma)\,A^{-1}(\sigma)\,\boldsymbol{\varphi} \\
&= -N + \sigma^{-2} \left[ \text{tr}\, A^{-1}(\sigma)
+ \boldsymbol{\varphi}^t A^{-2}(\sigma)\,\boldsymbol{\varphi}\right] \; ,
\end{aligned}
$$
using
$$
\frac{d}{d\sigma} A(\sigma) = -2 \sigma^{-3} \mathbb{1}_N \; .
$$

The second approach involves...